# Web3 Scripts for Django
This notebook is used as my development environment for the scripts I intend to integrate with the LUCE Frontend. I prototype the functions here and then migrate them into script files to be used in conjunction with Django.


## First iteration:

### Prepare Connection to Blockchain


In [10]:
from web3 import Web3

In [11]:
!which python

/home/vagrant/miniconda/envs/luce_vm/bin/python


In [12]:
# Use Ganache as web3 instance
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))

### Create Ethereum Account

**Interactive**

In [14]:
# Create a new account
eth_account = w3.eth.account.create()

In [18]:
# Display address
eth_address = eth_account.address
eth_account.address

'0x3094C8bC5fb931c5734573f2AF729ef085a4043a'

In [19]:
# Show private key
eth_pkey = eth_account.privateKey
eth_account.privateKey

HexBytes('0x072e1c67f3828c312ba8915bc6d578b4bb5700922460611f0d7a5b00fe3726ad')

In [20]:
# Show balance
w3.eth.getBalance(eth_address)

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8545): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f5493b93278>: Failed to establish a new connection: [Errno 111] Connection refused'))

**Function**  
```
Inputs: -  
Output: account object (public key & private key)
```

In [69]:
# Create class to store address and private key
class EthAccount():
    address = None
    pkey = None

In [63]:
def create_wallet():
    eth_account = EthAccount()
    eth_account_raw = w3.eth.account.create()
    eth_account.address = eth_account_raw.address
    eth_account.pkey = eth_account_raw.privateKey
    return (eth_account)

In [66]:
eth_account = create_wallet()

### Fund Account

In [67]:
# Extract default accounts created by ganache
accounts = w3.eth.accounts

In [68]:
accounts

['0x92D44e8579620F2Db88A12E70FE38e8CDB3541BA',
 '0xbf7ca7e607516Cd0F1b94a6C77F45fbFD936949C',
 '0x43e196C418b4b7Ebf71ba534042cC8907bd39dc9',
 '0x2aD1e0fAC9d6D523Fb1f21aE1Bc6acA09dD60c7F',
 '0x3c93550555cDD5218bB2923008b3Fb4ea6Ee4618',
 '0xf12C911960968c187090b52998a3597b35BC2619',
 '0x172bE1ee3f659A83a9C307fafafA51c9F91954Ae',
 '0x515f90440fA63377d7451E58B3be334F9393Ad74',
 '0x27F82fD9E21a3758737Cf3BCCA9652064186107C',
 '0x6eEA265cd899aE17Cf588744E4C7Fe31e3258E85']

**Store credentials of first Ganache account to use as faucet**

In [70]:
# Instantiate faucet object
faucet = EthAccount()

In [71]:
# Wallet address
faucet.address       = "0x92D44e8579620F2Db88A12E70FE38e8CDB3541BA"

# Private key (from Ganache interface)
faucet.pkey   = "0x4a2cb86c7d3663abebf7ab86a6ddc3900aee399750f35e65a44ecf843ec39116"

**Send Function**

In [77]:
# Define a function to send ether
import time
def send_ether(amount_in_ether, recipient_address, sender_address = faucet.address, sender_pkey=faucet.pkey):
    amount_in_wei = w3.toWei(amount_in_ether,'ether');

    # How many transactions have been made by wallet?
    # This is required and prevents double-spending.
    # Different from nonce in block mining.
    nonce = w3.eth.getTransactionCount(sender_address)
    
    # Specify transcation dictionary
    txn_dict = {
            'to': recipient_address,
            'value': amount_in_wei,
            'gas': 2000000,
            'gasPrice': w3.toWei('40', 'gwei'),
            'nonce': nonce,
            'chainId': 3
    }
    
    # Sign transaction
    signed_txn = w3.eth.account.signTransaction(txn_dict, sender_pkey)

    # Send transaction & store transaction hash
    txn_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)

    # Check if transaction was added to blockchain
    # time.sleep(0.5)
    txn_receipt = w3.eth.getTransactionReceipt(txn_hash)
    return txn_hash

In [78]:
# Set recipient
recipient = eth_account.address

In [79]:
# Send ether and store transaction hash
txn_hash = send_ether(1.5,recipient)

In [83]:
# Show balance
w3.eth.getBalance(eth_account.address)

1500000000000000000

## Deploy Contract

### Compile contract from file

In [86]:
# Import libraries
#import json
#import web3

#from web3 import Web3
from solcx import compile_source
#from web3.contract import ConciseContract

Using solc version v0.4.25


In [87]:
# Read in LUCE contract code
with open('./data/luce.sol', 'r') as file:
    contract_source_code = file.read()

In [88]:
# Compile & Store Compiled source code
compiled_sol = compile_source(contract_source_code)

In [89]:
# Extract full interface as dict from compiled contract
contract_interface = compiled_sol['<stdin>:Dataset']

In [90]:
# Extract abi and bytecode
abi = contract_interface['abi']
bytecode = contract_interface['bin']

### Deploy

In [91]:
w3.eth.accounts[0]

'0x92D44e8579620F2Db88A12E70FE38e8CDB3541BA'

In [92]:
# Set sender
w3.eth.defaultAccount = eth_account.address

The default `eth.defaultAccount` address is used as the default "from" property for transaction dictionaries if no other explicit "from" property is specified.

In [93]:
# Create contract blueprint
Luce = w3.eth.contract(abi=abi, bytecode=bytecode)

---

w3.version.api

At this moment we are unable to use a privately created fresh ethereum account for contract deployment.. When trying to deploy a contract with a new account we receive the error:
```
ValueError: {'message': 'sender account not recognized'
```
--> Researched and no easy way to use fresh accounts for contract deployment on Ganache (it IS possible to import existing accounts into Ganache to be used. But the whole idea is to create new accounts on the fly..)  
--> Use pre-funded Ganache accounts instead and allocate them to new users.  
--> **UDATE:** It is possible and working now (see Third Iteration). But still might not be wise to implement due to security concerns. 

Three paths: 1) Ganache Pre-funded 2) Locally Hosted Wallets managed by server (pkey exposed e.g. in Django user model) 3) Wallets hosted on client side (requires JS, metamask and more effort on side of user)

### Obtain Transcation Receipt

In [41]:
# Wait for the transaction to be mined, and get the transaction receipt
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

In [42]:
# Obtain address of freshly deployed contract
tx_receipt.contractAddress

'0x1Fe1b20f88e305d4F993Fb67457BcE7863D267f2'

## Second iteration:
Instead of creating new accounts for each user we will allocate a pre-funded ganache account to each user instead..

I want access to the Django user model from within the jupyter environment.  
To achieve that I used [this](https://stackoverflow.com/questions/35483328/how-to-setup-jupyter-ipython-notebook-for-django) information.  
Before proceeding, ensure the `luce_vm` kernel is selected.

### Prepare Connection to Blockchain


In [1]:
from web3 import Web3

# Use Ganache as web3 instance
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))
accounts = w3.eth.accounts
# Save default accounts


### Get Access to Django user model

In [2]:
# Initialise Django context
import django
django.setup()

In [3]:
# Obtain user model
from django.contrib.auth import get_user_model
User = get_user_model()

### Obtain current user

In [4]:
user_count = len(User.objects.all())
idx = user_count-1
#current_user = request.user
current_user = User.objects.all()[0]

### Assign ethereum account

In [5]:
current_user.ethereum_public_key = accounts.pop()

In [6]:
# More general:
# current_user.ethereum_public_key = accounts[idx]

**Function with request being passed in**  
In this case the function handles the extraction of the user.

In [7]:
def assign_address(request):
    # Establish web3 connection
    from web3 import Web3
    w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))
    accounts = w3.eth.accounts
    # Obtain user model
    from django.contrib.auth import get_user_model
    User = get_user_model()
    # Obtain user count
    user_count = len(User.objects.all())
    idx = user_count-1
    # Assign web3 account to user
    current_user = request.user
    current_user.ethereum_public_key = accounts[idx]
    current_user.save()
    # Return user with address associated
    return current_user

**Function with user being passed in**  
In this case view.py handles extraction of relevant user.

In [8]:
def assign_address(user):
    # Establish web3 connection
    from web3 import Web3
    w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))
    accounts = w3.eth.accounts
    # Obtain user model
    from django.contrib.auth import get_user_model
    User = get_user_model()
    # Obtain user count
    user_count = len(User.objects.all())
    idx = user_count-1
    # Assign web3 account to user
    current_user = user
    current_user.ethereum_public_key = accounts[idx]
    current_user.save()
    # Return user with address associated
    return current_user

---

In [9]:
idx = len(User.objects.all())-1

In [10]:
idx

6

In [11]:
current_user = User.objects.all()[idx]

In [12]:
current_user

<User: test9@luce.com>

In [13]:
current_user.ethereum_public_key

'0xf12C911960968c187090b52998a3597b35BC2619'

In [14]:
current_user.staff = True
current_user.save()

---

### Deploy Contract during Dataset Upload

In [21]:
# obtain current user from Django view context
# current_user = user

### Compile contract

In [22]:
# Read in LUCE contract code
with open('./data/luce.sol', 'r') as file:
    contract_source_code = file.read()

In [23]:
# Compile & Store Compiled source code
compiled_sol = compile_source(contract_source_code)

In [24]:
# Extract full interface as dict from compiled contract
contract_interface = compiled_sol['<stdin>:Dataset']

In [25]:
# Extract abi and bytecode
abi = contract_interface['abi']
bytecode = contract_interface['bin']

All in one:

In [26]:
from solcx import compile_source

# Read in LUCE contract code
with open('./data/luce.sol', 'r') as file:
    contract_source_code = file.read()

# Compile & Store Compiled source code
compiled_sol = compile_source(contract_source_code)

# Extract full interface as dict from compiled contract
contract_interface = compiled_sol['<stdin>:Dataset']

# Extract abi and bytecode
abi = contract_interface['abi']
bytecode = contract_interface['bin']

### Deploy

In [27]:
from web3 import Web3

In [28]:
# Establish web3 connection
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))

In [29]:
# Set sender
w3.eth.defaultAccount = current_user.ethereum_public_key

The default `eth.defaultAccount` address is used as the default "from" property for transaction dictionaries if no other explicit "from" property is specified.

In [30]:
# Create contract blueprint
Luce = w3.eth.contract(abi=abi, bytecode=bytecode)

In [31]:
# Submit the transaction that deploys the contract
tx_hash = Luce.constructor().transact()

All in one:

In [32]:
# Establish web3 connection
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))

# Set sender
w3.eth.defaultAccount = current_user.ethereum_public_key

# Create contract blueprint
Luce = w3.eth.contract(abi=abi, bytecode=bytecode)

# Submit the transaction that deploys the contract
tx_hash = Luce.constructor().transact()

### Obtain Contract Receipt & Address

In [33]:
# Wait for the transaction to be mined, and get the transaction receipt
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

In [34]:
# Obtain address of freshly deployed contract
contract_address = tx_receipt.contractAddress
tx_receipt.contractAddress

'0xbf8A0e6404e28EfbD79B15859273d3E879F11f8e'

Store contract address together with dataset.  
If Dataset is updated, create a new contract and update address..

### Populate contract:

In [35]:
# Create python instance of deployed contract
luce = w3.eth.contract(
    address=contract_address,
    abi=contract_interface['abi'],
)

**publishData**  
```solidity
publishData(string memory _newdescription, string memory _link, uint _licence)
```

In [36]:
description = "description"
link = ""
license = 3
# Store dataset information in contract
tx_hash = luce.functions.publishData(description, link, license).transact()

In [37]:
contract_address

'0xbf8A0e6404e28EfbD79B15859273d3E879F11f8e'

**Full Function:**

In [38]:
def deploy_and_assign_contract(user):
    from solcx import compile_source
    from web3 import Web3
    
    # Read in LUCE contract code
    with open('./data/luce.sol', 'r') as file:
        contract_source_code = file.read()
        
    # Compile & Store Compiled source code
    compiled_sol = compile_source(contract_source_code)

    # Extract full interface as dict from compiled contract
    contract_interface = compiled_sol['<stdin>:Dataset']

    # Extract abi and bytecode
    abi = contract_interface['abi']
    bytecode = contract_interface['bin']
    
    # Establish web3 connection
    w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))

    # Obtin user
    current_user = user
    
    # Set sender
    w3.eth.defaultAccount = current_user.ethereum_public_key

    # Create contract blueprint
    Luce = w3.eth.contract(abi=abi, bytecode=bytecode)

    # Submit the transaction that deploys the contract
    tx_hash = Luce.constructor().transact()
    
    # Wait for the transaction to be mined, and get the transaction receipt
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    
    # Obtain address of freshly deployed contract
    contract_address = tx_receipt.contractAddress
    return contract_address

**Full code including feeding in of dataset information:**

In [39]:
def deploy_contract_with_data(user, description, license, link=""):
    from solcx import compile_source
    from web3 import Web3
    
    # Read in LUCE contract code
    with open('./data/luce.sol', 'r') as file:
        contract_source_code = file.read()
        
    # Compile & Store Compiled source code
    compiled_sol = compile_source(contract_source_code)

    # Extract full interface as dict from compiled contract
    contract_interface = compiled_sol['<stdin>:Dataset']

    # Extract abi and bytecode
    abi = contract_interface['abi']
    bytecode = contract_interface['bin']
    
    # Establish web3 connection
    w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))

    # Obtin user
    current_user = user
    
    # Set sender
    w3.eth.defaultAccount = current_user.ethereum_public_key

    # Create contract blueprint
    Luce = w3.eth.contract(abi=abi, bytecode=bytecode)

    # Submit the transaction that deploys the contract
    tx_hash = Luce.constructor().transact()
    
    # Wait for the transaction to be mined, and get the transaction receipt
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    
    # Obtain address of freshly deployed contract
    contract_address = tx_receipt.contractAddress
    
    # Create python instance of deployed contract
    luce = w3.eth.contract(
    address=contract_address,
    abi=contract_interface['abi'],
    )
    
    # Store dataset information in contract
    tx_hash = luce.functions.publishData(description, link, license).transact()
    
    return contract_address

---

### Interact with Contract
The next step is for a data requester to confirm that they agree with the licencing terms before being able to download a dataset.

In [160]:
from web3 import Web3

In [161]:
# Establish web3 connection
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))

In [162]:
# Set sender
w3.eth.defaultAccount = current_user.ethereum_public_key

---

The following functions are all to be executed by a potential data requestor:

First data requester checks the license.  
We perform a check that licence on smart contract indeed matches with licence displayed on web interface. (Since the one stored in the smart contract is auditable and binding. That said in practice they will always align.)

**getLicence**
```solidity
getLicence() public view returns(uint)
```

In [165]:
# Get Licence
luce.functions.getLicence().call()

3

Then, if he agrees to the licence, the data requestor adds himself to the dataset.  
* Include button on interface "I AGREE TO ABIDE BY THE LICENSING TERMS"
* Only once this agreement is made can they access the download

**addDataRequester**
```solidity
addDataRequester(uint purposeCode, uint licenceType) public returns(uint)
```

In [176]:
# Data Requester adds themself to the dataset (after confirmation)
luce.functions.addDataRequester(1,3).transact()

HexBytes('0x30b10084ec935e722077fc8b4e5c0401a47db1994c10ac15f8af7ab22156412a')

After a while - or if the dataset has changed - the data requester needs to renew the token that grants them access to the dataset

**renewToken**
```solidity
renewToken(uint compliance) public returns(uint token)
```

In [177]:
luce.functions.renewToken(1).transact()

HexBytes('0x0ba584f4f1694ec81c4ef55dd30f45ed62be758cf5d111e20f47d5d7378888db')

---

**getDescription**

In [163]:
luce.functions.dataDescription().call()

'description'

---

## Third iteration:
Going back to locally hosted private keys. This approach is more difficult but by now I feel I have sufficient experience to implement it. This will make it easier to transition to hosted nodes (Infura) and using testnets at a later stage. So it is worth investing the effort now while I have a good familiarty with the web3 libraries. In a few months it might take me a while to rech this level of familiarity with the framework again..

When transitioning to Infura at a later stage only the faucet needs to be adapted since then we can't make use of the Ganache pre-funded accounts anymore.

### Prepare Connection to Blockchain


In [37]:
from web3 import Web3

In [38]:
# Use Ganache as web3 instance
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))

### Create New Ethereum Account

In [39]:
# Create a new account
eth_account = w3.eth.account.create()
eth_account

In [40]:
# Display address
eth_address = eth_account.address
eth_account.address

'0xD7C6c4f9CF6eEb3d70b395954dd90E8478114EB3'

In [41]:
# Show private key
eth_pkey = eth_account.privateKey
eth_account.privateKey

HexBytes('0x0a998162ff5ecf96da414fcaf256e2a5094ba940b4a2adfea36741cc225bf137')

In [42]:
# Show balance
w3.eth.getBalance(eth_address)

0

**Function**

In [188]:
def create_wallet():
    eth_account = w3.eth.account.create()
    return (eth_account)
eth_account = create_wallet()

### HexBytes and Strings

I make use of the `eth_account` class to store ethereum accounts. `eth_account` objects internally use strings for the address attribute and HexBytes for the private key. So in order to comfortably interact with eth_account objects we want a way to convert back and forth between HexBytes and string.

In [189]:
type(eth_account)

eth_account.signers.local.LocalAccount

**Convert HexByte to string**

In [190]:
type(eth_account.privateKey)

hexbytes.main.HexBytes

In [191]:
type(eth_account.privateKey.hex())

str

In [192]:
string = eth_account.privateKey.hex()
eth_account.privateKey.hex()

'0x47683baba9f557dd1ecf0b91f7104995d356a0440460d6ae4b14182dbff6197b'

**Convert a string to HexByte**

In [193]:
from hexbytes import HexBytes

In [194]:
hb = HexBytes(string)

In [195]:
hb

HexBytes('0x47683baba9f557dd1ecf0b91f7104995d356a0440460d6ae4b14182dbff6197b')

### Fund Account

In [43]:
# Extract default accounts created by ganache
accounts = w3.eth.accounts[0:3]

In [44]:
accounts

['0x92D44e8579620F2Db88A12E70FE38e8CDB3541BA',
 '0xbf7ca7e607516Cd0F1b94a6C77F45fbFD936949C',
 '0x43e196C418b4b7Ebf71ba534042cC8907bd39dc9']

**Use first default Ganache account as faucet**

In [45]:
# Private key (from Ganache interface)
faucet_privateKey   = "0x4a2cb86c7d3663abebf7ab86a6ddc3900aee399750f35e65a44ecf843ec39116"

In [46]:
# Create faucet
faucet = w3.eth.account.privateKeyToAccount(faucet_privateKey)

In [47]:
# Verify that address matches
print(accounts[0])
print(w3.eth.coinbase)
print(faucet.address)

0x92D44e8579620F2Db88A12E70FE38e8CDB3541BA
0x92D44e8579620F2Db88A12E70FE38e8CDB3541BA
0x92D44e8579620F2Db88A12E70FE38e8CDB3541BA


In [48]:
# Check balance
w3.eth.getBalance(faucet.address)

99399957811860000000000

**Send Function**

In [49]:
# Define function to send ether
import time
def send_ether(amount_in_ether, recipient_address, sender_pkey=faucet.privateKey):
    amount_in_wei = w3.toWei(amount_in_ether,'ether');
    
    # Obtain sender address from private key
    sender_address = w3.eth.account.privateKeyToAccount(sender_pkey).address

    # How many transactions have been made by wallet?
    # This is required and prevents double-spending.
    # Same name but different from nonce in block mining.
    nonce = w3.eth.getTransactionCount(sender_address)
    
    # Specify transcation dictionary
    txn_dict = {
            'to': recipient_address,
            'value': amount_in_wei,
            'gas': 2000000,
            'gasPrice': w3.toWei('40', 'gwei'),
            'nonce': nonce,
            'chainId': 3
    }
    
    # IN THIS STEP THE PRIVATE KEY OF THE SENDER IS USED
    # Sign transaction
    def sign_transaction(txn_dict, sender_pkey):
        signed_txn = w3.eth.account.signTransaction(txn_dict, sender_pkey)
        return signed_txn
    signed_txn      = sign_transaction(txn_dict, sender_pkey)
    signed_txn_raw = signed_txn.rawTransaction
    
    
    # Send transaction & store transaction hash
    def send_raw_transaction(raw_transaction):
        txn_hash = w3.eth.sendRawTransaction(raw_transaction)
        return txn_hash
    txn_hash = send_raw_transaction(signed_txn_raw)

    # Check if transaction was added to blockchain 
    # time.sleep(5)     # Not needed on Ganache since our transactions are instantaneous
    txn_receipt = w3.eth.getTransactionReceipt(txn_hash)
    
#     if txn_receipt:
#         return True # The transaction was successful
    return txn_hash

In [50]:
# Set recipient
recipient = eth_account.address

In [51]:
# Check balance before transfer
w3.eth.getBalance(eth_account.address)

0

In [52]:
# Send ether and store transaction hash
txn_hash = send_ether(10,recipient)

In [57]:
# Show balance
w3.eth.getBalance(eth_account.address)

10000000000000000000

**Create Account and Pre-Fund**  
Function to be used in Django when a new user registers

*Django Context*

In [54]:
# Initialise Django context
import django
django.setup()

In [55]:
# Obtain user model
from django.contrib.auth import get_user_model
User = get_user_model()

In [56]:
#current_user = request.user
user = User.objects.all()[1]

In [285]:
user

<User: test@luce.com>

In [248]:
# Old function for reference
def assign_address(user):
    # Establish web3 connection
    from web3 import Web3
    w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))
    accounts = w3.eth.accounts
    # Obtain user model
    from django.contrib.auth import get_user_model
    User = get_user_model()
    # Obtain user count
    user_count = len(User.objects.all())
    idx = user_count-1
    # Assign web3 account to user
    current_user = user
    current_user.ethereum_public_key = accounts[idx]
    current_user.save()
    # Return user with address associated
    return current_user

In [257]:
def create_wallet():
    eth_account = w3.eth.account.create()
    return (eth_account)

In [258]:
# Required send_ether to be defined
def fund_wallet(recipient = eth_account.address, amount = 100):
    send_ether(amount,recipient)

In [256]:
eth_account.privateKey.hex()

'0x5f5db2dfb4f0d956583b78c447dd788514a75fd98410babb0f99e2b234b9c7dc'

In [289]:
# New function | requires create_wallet, fund_wallet and send_ether to be defined
def assign_address_v3(user):
    # Establish web3 connection
    from web3 import Web3
    import time
    from hexbytes import HexBytes
    w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))
    accounts = w3.eth.accounts
    current_user = user
    
    # Create new web3 account
    eth_account = create_wallet()
    
    # Store public key and private key in user model
    current_user.ethereum_public_key = eth_account.address
    current_user.ethereum_private_key = eth_account.privateKey.hex()
    current_user.save()
    
    # Fund wallet
    fund_wallet(recipient = eth_account.address, amount = 100)
    
    # Return user, now with wallet associated
    return current_user

**Test**

In [309]:
user

<User: test@luce.com>

In [310]:
user.ethereum_public_key

'0x1d6F9ff1f0A1267de88dB8eb52119659713b614f'

In [311]:
user.ethereum_private_key

'0xedeeca4ee4c85b6a882aebd72ac61a4283e3f2cdb37f9339711253ff000ef7de'

In [312]:
bal = w3.eth.getBalance(user.ethereum_public_key)
w3.eth.getBalance(user.ethereum_public_key)

100000000000000000000

In [313]:
# Show balance in ether
w3.fromWei(bal, 'ether')

Decimal('100')

In [314]:
assign_address_v3(user)

<User: test@luce.com>

In [315]:
user.ethereum_public_key

'0xB617c680bB74E80b673e99230FDB8cC3174e3f32'

In [316]:
print(w3.fromWei(w3.eth.getBalance(user.ethereum_public_key), 'ether'))

100


Yes, all works as intended.  
Function takes in a user, generates a new wallet, stores the address (and privatekey) and gives some ether to the account.

### Deploy Contract

### Compile contract from file

In [58]:
from solcx import compile_source

In [59]:
# Read in LUCE contract code
with open('./data/luce.sol', 'r') as file:
    contract_source_code = file.read()

In [60]:
# Compile & Store Compiled source code
compiled_sol = compile_source(contract_source_code)

In [61]:
# Extract full interface as dict from compiled contract
contract_interface = compiled_sol['<stdin>:Dataset']

In [62]:
# Extract abi and bytecode
abi = contract_interface['abi']
bytecode = contract_interface['bin']

In [63]:
# Contract object
my_contract   = w3.eth.contract(abi=abi, bytecode=bytecode)

In [64]:
type(my_contract.bytecode)

hexbytes.main.HexBytes

In [65]:
# Contract data
contract_data = bytecode

In [66]:
# Construct raw transaction
nonce = w3.eth.getTransactionCount(eth_account.address)
transaction = {
    'from': eth_account.address,
    'gas': 2000000,
    'data': bytecode,
    'chainId': 3,
    'gasPrice': w3.toWei('40', 'gwei'),
    'nonce': nonce,
}

In [67]:
# Sign transaction
signed_txn = w3.eth.account.signTransaction(transaction, eth_account.privateKey)

In [68]:
# Deploy
w3.eth.sendRawTransaction(signed_txn.rawTransaction)

HexBytes('0xaf22fbf606a4247a36561814fa8883164946abd295c16e75cc9337c619c33e8e')

**Hallelujah, this worked! :D**  
Note to self: It took me many, many days to get to this point.  
And even today I spent literally four hours doing nothing but trying to get this to work since no documentation/examples/stackoverflow questions exist. And here we go, it is THAT simple. (As always, once we learn how to properly do it..) I am very happy.  

Now we are able to deploy contracts using locally created keys. This was the missing piece to enable a lot of other applications. Now we can create unlimited local accounts for testing on the fly and simply pre-fund them from an arbitrarily wealthy default account. Also transition to Infura is easy now.

---

Proper approach:

In [131]:
# Create contract object
Luce = w3.eth.contract(abi=abi, bytecode=bytecode)

Using pre-funded account the deployment is easy:

In [132]:
Luce.constructor().transact({'from':w3.eth.coinbase})

HexBytes('0x93b7efbf6a1cd8b85497c10f361d36c51090079d0749d27b602b59d32df885a1')

To use a local account we have to manually build the transaction that deploys the contract, sign it and then transmit it as a (signed) raw transaction:

In [139]:
# Construct raw transaction
nonce = w3.eth.getTransactionCount(eth_account.address)
transaction = {
    'from': eth_account.address,
    'gas': 2000000,
    'data': bytecode,
    'chainId': 3,
    'gasPrice': w3.toWei('40', 'gwei'),
    'nonce': nonce,
}

In [140]:
# Sign transaction
signed_txn = w3.eth.account.signTransaction(transaction, eth_account.privateKey)

In [141]:
# Deploy
tx_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)

In [ ]:
# Wait for the transaction to be mined, and get the transaction receipt
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
  

In [143]:
tx_receipt

AttributeDict({'transactionHash': HexBytes('0x45ce9baf68d5665f9bd72b881c32a27783ffc39383f42b2e7cb246c7b61360e3'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0xe8c8891377fcc83b58d93fc15373d5de74d78bfaa31d20be4c910a2ea33bd6d7'),
 'blockNumber': 44,
 'from': '0xd7c6c4f9cf6eeb3d70b395954dd90e8478114eb3',
 'to': None,
 'gasUsed': 1338398,
 'cumulativeGasUsed': 1338398,
 'contractAddress': '0xDFd09525070497ec3d2086456Cf1533A45A06729',
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [142]:
  # Obtain address of freshly deployed contract
tx_receipt.contractAddress

'0xDFd09525070497ec3d2086456Cf1533A45A06729'

**Function**

In [333]:
def deploy_contract_v3(private_key):
    from solcx import compile_source
    from web3 import Web3
    
    # Read in LUCE contract code
    with open('./data/luce.sol', 'r') as file: # Adjust file_path for use in Jupyter/Django
        contract_source_code = file.read()
    
    # Compile & Store Compiled source code
    compiled_sol = compile_source(contract_source_code)

    # Extract full interface as dict from compiled contract
    contract_interface = compiled_sol['<stdin>:Dataset']

    # Extract abi and bytecode
    abi = contract_interface['abi']
    bytecode = contract_interface['bin']
    
    # Establish web3 connection
    w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))
    
    #Obtain user so we know his address for the 'from' field
    user = w3.eth.account.privateKeyToAccount(private_key)
    
    # Construct raw transaction
    nonce = w3.eth.getTransactionCount(user.address)
    
    transaction = {
    'from': user.address,
    'gas': 2000000,
    'data': bytecode,
    'chainId': 3,
    'gasPrice': w3.toWei('40', 'gwei'),
    'nonce': nonce,
    }
    
    # Sign transaction
    signed_txn = w3.eth.account.signTransaction(transaction, private_key)
    
    # Deploy
    tx_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
    
    # Wait for the transaction to be mined, and get the transaction receipt
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    
    # Obtain address of freshly deployed contract
    contractAddress = tx_receipt.contractAddress
    
    return contractAddress

**Test**

In [334]:
user.ethereum_private_key

'0x0f30fd50e14582db06de9643442e3f98d8ce9094185a9769dadea907828ed19d'

In [335]:
deploy_contract_v3(user.ethereum_private_key)

'0xB3881c4C36E8607BD128724FF56170ae662aE942'

In [340]:
bal = w3.eth.getBalance(faucet.address)

In [341]:
w3.fromWei(bal, 'ether')

Decimal('100000')

---

### Publish Data
Now that the smart contract is deployed information about the dataset needs to be added

In [91]:
luce.functions.publishData("test", "link", 3).transact()

ValueError: {'message': 'from not found; is required', 'code': -32000, 'data': {'stack': 'n: from not found; is required\n    at _.queueTransaction (/home/vagrant/.nvm/versions/node/v12.6.0/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:52:520373)\n    at u.eth_sendTransaction (/home/vagrant/.nvm/versions/node/v12.6.0/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:52:510635)\n    at u.handleRequest (/home/vagrant/.nvm/versions/node/v12.6.0/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:52:507798)\n    at t (/home/vagrant/.nvm/versions/node/v12.6.0/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:52:371362)\n    at a.handleRequest (/home/vagrant/.nvm/versions/node/v12.6.0/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:52:506162)\n    at t (/home/vagrant/.nvm/versions/node/v12.6.0/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:52:371362)\n    at s.f.handleRequest (/home/vagrant/.nvm/versions/node/v12.6.0/lib/node

In [96]:
def compile_and_extract_interface():
    from solcx import compile_source
    
    # Read in LUCE contract code
    with open('./data/luce.sol', 'r') as file: # Adjust file_path for use in Jupyter/Django
        contract_source_code = file.read()
        
    # Compile & Store Compiled source code
    compiled_sol = compile_source(contract_source_code)

    # Extract full interface as dict from compiled contract
    contract_interface = compiled_sol['<stdin>:Dataset']

    # Extract abi and bytecode
    abi = contract_interface['abi']
    bytecode = contract_interface['bin']
    
    # Create dictionary with interface
    d = dict()
    d['abi']      = abi
    d['bytecode'] = bytecode
    d['full_interface'] = contract_interface
    return(d)

In [97]:
def publish_data_v3(provider_private_key, contract_address, description="Description", license=3, link="void"):
    from web3 import Web3
    
    # Compile Luce contract and obtain interface
    d = compile_and_extract_interface()
    
    # Establish web3 connection
    w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))
    
    # Obtain user so we know his address for the 'from' field
    private_key = provider_private_key
    user = w3.eth.account.privateKeyToAccount(private_key)
    
    # Obtain contract address & instantiate contract
    contract_address = contract_address
    luce = w3.eth.contract(address=contract_address, abi=d['abi'])
    
    # Construct raw transaction
    nonce = w3.eth.getTransactionCount(user.address)
    txn_dict = {
    'gas': 2000000,
    'chainId': 3,
    'gasPrice': w3.toWei('40', 'gwei'),
    'nonce': nonce,
    }
    
    luce_txn = luce.functions.publishData(description,link,license).buildTransaction(txn_dict)
    
    # Sign transaction
    signed_txn = w3.eth.account.signTransaction(luce_txn, private_key)
    
    # Deploy
    tx_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
    
    # Wait for the transaction to be mined, and get the transaction receipt
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    
    # Obtain address of freshly deployed contract
    contractAddress = tx_receipt.contractAddress
    
    return contractAddress

In [98]:
publish_data_v3(eth_account.privateKey, test_contract_address)

---

### Request Access to Dataset
Here we will add a user to the list of verified data requesters of the smart contract. (And simulatneously store the access right of that user in the DB connected to Django.)

Needed:
* address of smart contract representing the desired dataset
    * data.contract_address
* instance of the contract in python 
    * obtain with ABI and contract address
* private key of data requestor to sign transaction

STEPS:  
    1. Create contract instance  
    2) Obtain privKey of Data Requestor  
    3) Build transaction 
    4) Sign & Send transaction

**addDataRequester**
```solidity
addDataRequester(uint purposeCode, uint licenceType) public returns(uint)
```

In [ ]:
luce1.functions.addDataRequester(1,5).transact()

**Function**

In [3]:
d = dict()
d["abi"] = "test"

In [4]:
d

{'abi': 'test'}

In [35]:
def compile_and_extract_interface():
    from solcx import compile_source
    
    # Read in LUCE contract code
    with open('./data/luce.sol', 'r') as file: # Adjust file_path for use in Jupyter/Django
        contract_source_code = file.read()
        
    # Compile & Store Compiled source code
    compiled_sol = compile_source(contract_source_code)

    # Extract full interface as dict from compiled contract
    contract_interface = compiled_sol['<stdin>:Dataset']

    # Extract abi and bytecode
    abi = contract_interface['abi']
    bytecode = contract_interface['bin']
    
    # Create dictionary with interface
    d = dict()
    d['abi']      = abi
    d['bytecode'] = bytecode
    d['full_interface'] = contract_interface
    return(d)

In [28]:
d = compile_and_extract_interface()

In [29]:
d.keys()

dict_keys(['abi', 'bytecode', 'full_interface'])

In [15]:
type(d['abi'])

list

In [32]:
type(d['bytecode'])

str

In [33]:
type(d['full_interface'])

dict

In [99]:
def add_requester_v3(requester_private_key, contract_address, license_type=3, purpose_code=3):
    from web3 import Web3
    
    # Compile Luce contract and obtain interface
    d = compile_and_extract_interface()
    
    # Establish web3 connection
    w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))
    
    # Obtain user so we know his address for the 'from' field
    private_key = requester_private_key
    user = w3.eth.account.privateKeyToAccount(private_key)
    
    # Obtain contract address & instantiate contract
    contract_address = contract_address
    luce = w3.eth.contract(address=contract_address, abi=d['abi'])
    
    # Construct raw transaction
    nonce = w3.eth.getTransactionCount(user.address)
    txn_dict = {
    'gas': 2000000,
    'chainId': 3,
    'gasPrice': w3.toWei('40', 'gwei'),
    'nonce': nonce,
    }
    
    luce_txn = luce.functions.addDataRequester(purpose_code,license_type).buildTransaction(txn_dict)
    
    # Sign transaction
    signed_txn = w3.eth.account.signTransaction(luce_txn, private_key)
    
    # Deploy
    tx_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
    
    # Wait for the transaction to be mined, and get the transaction receipt
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    
    # Obtain address of freshly deployed contract
    contractAddress = tx_receipt.contractAddress
    
    return contractAddress

In [100]:
add_requester_v3(eth_account.privateKey, test_contract_address)

In [36]:
from web3 import Web3
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))

In [118]:
test_contract_address = Web3.toChecksumAddress('0x773f194d0551423ab67b76157d20ca7d467bceed')

In [119]:
luce = w3.eth.contract(address=test_address,abi=abi)

In [120]:
nonce = w3.eth.getTransactionCount(eth_account.address)

In [121]:
luce_txn = luce.functions.addDataRequester(1,3).buildTransaction({
    'from': eth_account.address,
    'gas': 2000000,
    'chainId': 3,
    'gasPrice': w3.toWei('40', 'gwei'),
    'nonce': nonce,
})

In [122]:
 # Sign transaction
signed_txn = w3.eth.account.signTransaction(luce_txn, eth_account.privateKey)

In [123]:
# Deploy
tx_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)

In [124]:
# Wait for the transaction to be mined, and get the transaction receipt
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

In [125]:
# Obtain address of freshly deployed contract
contractAddress = tx_receipt.contractAddress

In [130]:
tx_receipt

AttributeDict({'transactionHash': HexBytes('0x89f92ec8c375bacdef5c0d361818a2e88d2b83ded991a4d6d1e490caaa7b0465'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0xb43b4f5f8c89dda2a296f3a36ef2df9a3746ff9549812db63ef6f5726fbdf940'),
 'blockNumber': 41,
 'from': '0xd7c6c4f9cf6eeb3d70b395954dd90e8478114eb3',
 'to': '0x773f194d0551423ab67b76157d20ca7d467bceed',
 'gasUsed': 58442,
 'cumulativeGasUsed': 58442,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'

In [126]:
contractAddress

### Update Dataset
Here we update the data in the smart contract when a dataset is updated by its owner.

**updateData**
```solidity
updateData(string memory updateDescr, string memory _newlink)
```

In [1]:
# Update Data
_ = luce1.functions.updateData("New Description", "Void").transact()

NameError: name 'luce1' is not defined

In [108]:
def update_contract_v3(provider_private_key, contract_address, description="Updated Description", link="void"):
    from web3 import Web3
    
    # Compile Luce contract and obtain interface
    d = compile_and_extract_interface()
    
    # Establish web3 connection
    w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))
    
    # Obtain user so we know his address for the 'from' field
    private_key = provider_private_key
    user = w3.eth.account.privateKeyToAccount(private_key)
    
    # Obtain contract address & instantiate contract
    contract_address = contract_address
    luce = w3.eth.contract(address=contract_address, abi=d['abi'])
    
    # Construct raw transaction
    nonce = w3.eth.getTransactionCount(user.address)
    txn_dict = {
    'gas': 2000000,
    'chainId': 3,
    'gasPrice': w3.toWei('40', 'gwei'),
    'nonce': nonce,
    }
    
    luce_txn = luce.functions.updateData(description,link).buildTransaction(txn_dict)
    
    # Sign transaction
    signed_txn = w3.eth.account.signTransaction(luce_txn, private_key)
    
    # Deploy
    tx_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
    
    # Wait for the transaction to be mined, and get the transaction receipt
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    
    # Obtain address of freshly deployed contract
    contractAddress = tx_receipt.contractAddress
    
    return contractAddress

In [109]:
update_contract_v3(eth_account.privateKey, test_contract_address)

In [117]:
luce.functions.dataDescription().call()

'Updated Description'

## Playground | Scratchpad

### Get Access to Django user model

In [1]:
# Initialise Django context
import django
django.setup()

In [2]:
# Obtain user model
from django.contrib.auth import get_user_model
User = get_user_model()

In [3]:
User.objects.all()

<QuerySet [<User: vagrant@luce.com>, <User: test@luce.com>, <User: test5@luce.com>, <User: test6@luce.com>, <User: test7@luce.com>, <User: test8@luce.com>, <User: test9@luce.com>, <User: test2@luce.com>]>

---

In [4]:
u = User.objects.all()[1]

In [10]:
# Reload object
u.refresh_from_db()

In [6]:
u

<User: test@luce.com>

In [7]:
u.ethereum_public_key

'0x2804FfD3A4174A991B8f18d781887f19eed6335D'

In [8]:
u.ethereum_private_key

'0x94e0d07c56c7435896fd5d4ac4b9ad1ec4624ab676aa0fdbc1e7f7e40c5b0a87'